In [59]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate

# !pip install imbalanced-learn

from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

import pickle

In [60]:
data = pd.read_csv('CHLA_clean_data_until_2023.csv')

In [61]:
df = data[['LEAD_TIME', 'APPT_TYPE_STANDARDIZE' ,'APPT_NUM', 'TOTAL_NUMBER_OF_CANCELLATIONS', 'TOTAL_NUMBER_OF_RESCHEDULED', 'TOTAL_NUMBER_OF_NOT_CHECKOUT_APPOINTMENT',
    'TOTAL_NUMBER_OF_SUCCESS_APPOINTMENT', 'TOTAL_NUMBER_OF_NOSHOW', 'DAY_OF_WEEK', 'WEEK_OF_MONTH', 'NUM_OF_MONTH', 'HOUR_OF_DAY', 'IS_NOSHOW', 'AGE', 'ETHNICITY_STANDARDIZE',
    'RACE_STANDARDIZE', 'IS_REPEAT']]

In [62]:
label_encoder_dict = {}

columns_to_convert = ['APPT_TYPE_STANDARDIZE', 'IS_REPEAT', 'ETHNICITY_STANDARDIZE', 'RACE_STANDARDIZE', 'DAY_OF_WEEK', 'WEEK_OF_MONTH', 'NUM_OF_MONTH', 'HOUR_OF_DAY']

for col in columns_to_convert:
    label_encoder = LabelEncoder()
    df[col] = label_encoder.fit_transform(df[col])
    label_encoder_dict[col] = list(label_encoder.classes_)

C:\Users\trant\AppData\Local\Temp\ipykernel_18824\2400906684.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = label_encoder.fit_transform(df[col])
C:\Users\trant\AppData\Local\Temp\ipykernel_18824\2400906684.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = label_encoder.fit_transform(df[col])
C:\Users\trant\AppData\Local\Temp\ipykernel_18824\2400906684.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [63]:
# label_encoder = LabelEncoder()
# columns_to_convert = ['APPT_TYPE_STANDARDIZE', 'IS_REPEAT', 'ETHNICITY_STANDARDIZE', 'RACE_STANDARDIZE', 'DAY_OF_WEEK', 'WEEK_OF_MONTH', 'NUM_OF_MONTH', 'HOUR_OF_DAY']


# for col in columns_to_convert:
#     df.loc[:, col] = label_encoder.fit_transform(df[col])
    
    


In [64]:
features = df[['LEAD_TIME', 'APPT_TYPE_STANDARDIZE' ,'APPT_NUM', 'TOTAL_NUMBER_OF_CANCELLATIONS', 'TOTAL_NUMBER_OF_RESCHEDULED', 'TOTAL_NUMBER_OF_NOT_CHECKOUT_APPOINTMENT',
    'TOTAL_NUMBER_OF_SUCCESS_APPOINTMENT', 'TOTAL_NUMBER_OF_NOSHOW', 'DAY_OF_WEEK', 'WEEK_OF_MONTH', 'NUM_OF_MONTH', 'HOUR_OF_DAY', 'AGE', 'ETHNICITY_STANDARDIZE',
    'RACE_STANDARDIZE', 'IS_REPEAT']]

In [77]:
encoder_dict = {'APPT_TYPE_STANDARDIZE':['Follow-up','New','Others']}

#save a label encoder
encoder_pickle_out = open("label_encoder.pkl","wb")
pickle.dump(encoder_dict, encoder_pickle_out)
encoder_pickle_out.close()


In [66]:
print(encoder_dict)

{'APPT_TYPE_STANDARDIZE': ['Follow-up', 'New', 'Others']}


In [67]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(features, df['IS_NOSHOW'])

In [68]:
#encode the target
df.loc[:, 'IS_NOSHOW'] = label_encoder.fit_transform(df['IS_NOSHOW'])

In [69]:
scaler = MinMaxScaler()

scaled_features = scaler.fit_transform(X_resampled)

selector = SelectKBest(score_func=chi2, k=8)
X_selected = selector.fit_transform(scaled_features, y_resampled)

In [70]:
selected_indices = selector.get_support(indices=True)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_resampled, test_size = 0.3, random_state = 26, stratify = y_resampled)

In [72]:
# Want higher precision to ensure each patient can get a chance to get an appointment

model_logistic = LogisticRegression()
model_logistic.fit(X_train, y_train)

y_pred = model_logistic.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,pos_label='Y')
recall = recall_score(y_test, y_pred,pos_label='Y')
f1 = f1_score(y_test, y_pred,pos_label='Y')

# Printing other metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9388
Precision: 0.9160
Recall: 0.9661
F1 Score: 0.9404


In [73]:
classifier = DecisionTreeClassifier(max_depth=3, random_state=42,max_features=None, min_samples_leaf=15)

# Fit the classifier to the training data
classifier.fit(X_train, y_train)

# Predict the test set results
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,pos_label='Y')
recall = recall_score(y_test, y_pred,pos_label='Y')
f1 = f1_score(y_test, y_pred,pos_label='Y')

# Printing other metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9383
Precision: 0.8980
Recall: 0.9890
F1 Score: 0.9413


In [74]:
model = GaussianNB()

model.fit(X_train, y_train)

preds = model.predict(X_test)

accuracy = metrics.accuracy_score(y_test, preds)

print(accuracy)

0.9234417907963718


In [78]:
#laoding model into a pickle file
pickle_out = open("model.pkl","wb")
pickle.dump(model, pickle_out)
pickle_out.close()


In [75]:
# coefficients = model_logistic.coef_
# intercept = model_logistic.intercept_

# # Save coefficients to a pickle file
# with open('logistic_model_coefficients.pkl', 'wb') as f:
#     pickle.dump((coefficients, intercept), f)

In [76]:
arr = [1, 2, 3, 4, 6, 7, 9, 14]

columns_to_display = [col for col in df.columns if df.columns.get_loc(col) + 1 in arr]
columns_to_display

['LEAD_TIME',
 'APPT_TYPE_STANDARDIZE',
 'APPT_NUM',
 'TOTAL_NUMBER_OF_CANCELLATIONS',
 'TOTAL_NUMBER_OF_NOT_CHECKOUT_APPOINTMENT',
 'TOTAL_NUMBER_OF_SUCCESS_APPOINTMENT',
 'DAY_OF_WEEK',
 'AGE']